In [3]:
from pathlib import Path
out_path_parquet = "/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-04/part-000.parquet"
df = con.execute(""" Select * from read_parquet(?) """,[str(out_path_parquet)]).df()
df.head()

,year,country_of_asylum,country_of_origin,refugees,returned_refugees,asylum_seekers,idps,returned_idps,stateless,hst,ooc,date
0,2020,AFG,AFG,0,0,0,2886317,0,0,0,82595,2025-09-04
1,2020,ALB,AFG,0,0,0,0,0,0,0,5,2025-09-04
2,2020,EGY,AFG,34,0,44,0,0,0,0,0,2025-09-04
3,2020,ARG,AFG,12,0,0,0,0,0,0,0,2025-09-04
4,2020,ARM,AFG,5,0,0,0,0,0,0,0,2025-09-04


In [9]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
from dis import disco
import os
import glob
from pathlib import Path
path = Path(r"/home/faacosta0245695/conflit/conflit_warehouse/data/bronze")
def discovery (path_warehouse: Path, chunk_size=8192):
    for root, dirs, files in os.walk(path_warehouse):
        for file in files:
            if file.endswith('.parquet'):
                print(root, file)
discovery(path)

/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-04 [] part-000.parquet


In [45]:
import os
import hashlib
from pathlib import Path
from datetime import datetime
def compute_file_hash(path: Path, chunk_size=65536) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while chunk := f.read(chunk_size):
            h.update(chunk)
    return h.hexdigest()

def discovery(path_warehouse: Path, chunk_size=65536):
    records = []
    for root, _, files in os.walk(path_warehouse):
        for file in files:
            if file.endswith(".parquet"):
                file_path = Path(root) / file
                records.append({
                    "file_path": str(file_path),
                    "content_hash": compute_file_hash(file_path, chunk_size),
                    "file_size": file_path.stat().st_size,
                    "discovered_at": datetime.now(),
                    "source_partition": next((p for p in file_path.parts if p.startswith("date=")), None)
                })
    return records

# Example usage
bronze_path = Path("/home/faacosta0245695/conflit/conflit_warehouse/data/bronze")
files = discovery(bronze_path)
print(files[:2])  # peek first two

[{'file_path': '/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-04/part-000.parquet', 'content_hash': '6ae7cf49a930577cc4a3f216f2e09539fe1b9e7eb7ac0731b820ebc8189acd8a', 'file_size': 31338, 'discovered_at': datetime.datetime(2025, 9, 8, 15, 54, 59, 285006), 'source_partition': 'date=2025-09-04'}]


In [3]:
import duckdb
def supports_merge(con: duckdb.DuckDBPyConnection) -> bool:
    try:
        con.execute("CREATE TEMP TABLE _a(x INT);")
        con.execute("CREATE TEMP TABLE _b(x INT);")
        con.execute("MERGE INTO _a a USING _b b ON a.x = b.x "
                    "WHEN MATCHED THEN UPDATE SET x=b.x "
                    "WHEN NOT MATCHED THEN INSERT VALUES (b.x);")
        return True
    except duckdb.Error:
        return False
    finally:
        con.execute("DROP TABLE IF EXISTS _a;")
        con.execute("DROP TABLE IF EXISTS _b;")
con = duckdb.connect()
supports_merge(con)

False

In [1]:
import duckdb 
con = duckdb.connect("/home/faacosta0245695/conflit/conflit_warehouse/warehouse/database.db")
con.execute("""Select * from dirty_partitions""").df()


,entity,partition_key,reason,first_seen_at,last_seen_at,status
0,refugee_displacement_conformed,2020,pending_manifest,2025-09-15 14:07:34.192,2025-09-15 14:07:34.192,dirty


In [2]:
import duckdb 
con = duckdb.connect("/home/faacosta0245695/conflit/conflit_warehouse/warehouse/database.db")
def planner(con):
    dirty = con.execute(""" SELECT entity, partition_key
                                FROM dirty_partitions
                                WHERE status = 'dirty'
                                ORDER BY last_seen_at, entity, partition_key; """).df()
    for entity, partition_key in zip(dirty['entity'],dirty['partition_key']):
        plan_df  = con.execute(f""" 
            WITH candidates AS (
                SELECT mpl.file_path, mpl.content_hash, m.file_size, m.discovered_at, m.status
                FROM manifest_partition_link mpl
                JOIN ingest_manifest m
                    ON m.file_path = mpl.file_path
                AND m.content_hash = mpl.content_hash
                WHERE mpl.entity = ?
                    AND mpl.partition_key = ?
                ),
                latest_per_path AS (
                SELECT * EXCLUDE rn
                FROM (
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY file_path ORDER BY discovered_at DESC) rn
                    FROM candidates
                    WHERE status IN ('pending','processed')
                ) WHERE rn = 1
                )
                SELECT * FROM latest_per_path
                ORDER BY discovered_at ASC, file_path ASC;
    
            """,[entity, partition_key]).df()
        if plan_df.empty:
            pass

        yield {
            "entity": entity,
            "partition_key": partition_key,
            "inputs": plan_df.to_dict("records"),
        }        
for i in planner(con):
    print(i)

{'entity': 'refugee_displacement_conformed', 'partition_key': '2020', 'inputs': [{'file_path': '/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-09/2020-2020-part-000.parquet', 'content_hash': '6ae7cf49a930577cc4a3f216f2e09539fe1b9e7eb7ac0731b820ebc8189acd8a', 'file_size': 31338, 'discovered_at': Timestamp('2025-09-09 11:55:06.614298'), 'status': 'pending'}]}


In [5]:
import pandas as pd
import yaml
from pathlib import Path

def load_schema(path: Path):
    with open(path, "r") as f:
        schema = yaml.safe_load(f)
    return schema
def harmonize(df_raw: pd.DataFrame, schema: dict) -> pd.DataFrame:
    rename_map = {}
    casts = {}
    for col in schema["columns"]:
        canonical = col["name"]
        found = next((c for c in col["source"] if c in df_raw.columns), None)
        if found:
            rename_map[found] = canonical
        casts[canonical] = col["type"]
    
    df = df_raw.rename(columns=rename_map)
    
    for col in [c["name"] for c in schema["columns"]]:
        if col not in df.columns:
            df[col] = None
    
    for col, typ in casts.items():
        if typ == "BIGINT":
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
        elif typ == "DOUBLE":
            df[col] = pd.to_numeric(df[col], errors="coerce")
        elif typ == "STRING":
            df[col] = df[col].astype("string")
        elif typ == "DATE":
            df[col] = pd.to_datetime(df[col], errors="coerce")
    
    for col in schema.get("lineage_columns", []):
        if col not in df.columns:
            df[col] = None
    
    df = df[[c["name"] for c in schema["columns"]] + schema.get("lineage_columns", [])]
    return df
raw = pd.read_csv("/home/faacosta0245695/conflit/conflit_warehouse/data/raw/unhcr/persons_of_concern.csv")
schema = load_schema("/home/faacosta0245695/conflit/conflit_warehouse/schemas/refugees_stack.yaml")

print(harmonize(raw,schema))

      country_origin country_destination  refugees  asylum_seekers     idps  \
0                AFG                 AFG         0               0  2553390   
1                AFG                 EGY        28              36        0   
2                AFG                 ARG        12               0        0   
3                AFG                 ARM         5               0        0   
4                AFG                 AUS     11585            1710        0   
...              ...                 ...       ...             ...      ...   
34706            ZWE                 SWE        24               0        0   
34707            ZWE                 CHE         9               5        0   
34708            ZWE                 USA       676            3037        0   
34709            ZWE                 ZMB        10              15        0   
34710            ZWE                 ZWE         0               0        0   

       stateless  hst     ooc  year _bronze_file _i